# Unit test writing using a multi-step prompt

Complex tasks, such as writing unit tests, can benefit from multi-step prompts. In contrast to a single prompt, a multi-step prompt generates text from GPT and then feeds that output text back into subsequent prompts. This can help in cases where you want GPT to reason things out before answering, or brainstorm a plan before executing it.

In this notebook, we use a 3-step prompt to write unit tests in Python using the following steps:

1. **Explain**: Given a Python function, we ask GPT to explain what the function is doing and why.
2. **Plan**: We ask GPT to plan a set of unit tests for the function.
    - If the plan is too short, we ask GPT to elaborate with more ideas for unit tests.
3. **Execute**: Finally, we instruct GPT to write unit tests that cover the planned cases.

The code example illustrates a few embellishments on the chained, multi-step prompt:

- Conditional branching (e.g., asking for elaboration only if the first plan is too short)
- The choice of different models for different steps
- A check that re-runs the function if the output is unsatisfactory (e.g., if the output code cannot be parsed by Python's `ast` module)
- Streaming output so that you can start reading the output before it's fully generated (handy for long, multi-step outputs)

In [57]:
import os
import openai  # used for calling the OpenAI API
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("OPENAI_API_KEY")

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = os.getenv("OPENAI_API_BASE")

# Currently Chat Completions API have the following versions available: 2023-03-15-preview
openai.api_version = os.getenv("OPENAI_API_VERSION")

chatgpt_deployment_id = "gpt-4"

color_prefix_by_role = {
    "system": "\033[0m",  # gray
    "user": "\033[0m",  # gray
    "assistant": "\033[92m",  # green
}


def print_messages(messages, color_prefix_by_role=color_prefix_by_role) -> None:
    """Prints messages sent to or from GPT."""
    for message in messages:
        role = message["role"]
        color_prefix = color_prefix_by_role[role]
        content = message["content"]
        print(f"{color_prefix}\n[{role}]\n{content}")


def print_message_delta(delta, color_prefix_by_role=color_prefix_by_role) -> None:
    """Prints a chunk of messages streamed back from GPT."""
    if "role" in delta:
        role = delta["role"]
        color_prefix = color_prefix_by_role[role]
        print(f"{color_prefix}\n[{role}]\n", end="")
    elif "content" in delta:
        content = delta["content"]
        print(content, end="")
    else:
        pass

def concat_chunks_in_response(response, print_text: bool = True) -> str:
    result = ""
    for chunk in response:
        delta = chunk["choices"][0]["delta"]
        if print_text:
            print_message_delta(delta)
        if "content" in delta:
            result += delta["content"]
    return result

In [68]:
def detailed_explanation_of_student_requirements(
    essay_content: str,  # Python function to test, as a string
    student_requirement:str, # student requirements, as a string
    explain_model: str = "gpt-3.5-turbo",
    temperature: float = 0.4,
    print_text: bool = False,  # optionally prints text; helpful for understanding the function & debugging
):
    """Returns a lesson plan for a given student's requirement and essay content, using a 3-step GPT prompt."""

# create a markdown-formatted message that asks GPT to explain the function, formatted as a bullet list
    explain_system_message = {
        "role": "system",
        "content": "You are a world-class Chinese teaching and research teacher with an eagle eye for learning objectives and student's deep-level requirements. You carefully anlysis learning objectives and student's specific requirements with great detail and accuracy. You organize your analysis in markdown-formatted, bulleted lists.",
    }
    explain_user_message = {
        "role": "user",
        "content": f"""Please explain learning objective and student's requirement in the following steps:
- Analyze the essay content.
- Explain the student's info.
- Try to conjecture student's existing knowledge in Chinese subject.
- Try to conjecture student's existing knowledge on given essay content.
- Analyze the student's potential deep-level requirements as much as possible based on previous steps.
- Define learning objectives: based on your above analysis, define clear and achievable learning objectives for this student.

Organize your explanation in English as a markdown-formatted, bulleted list.

# Input Parameters
## Essay Content
{essay_content}

## Student's Info
{student_requirement}
""",
    }
    explain_messages = [explain_system_message, explain_user_message]
    if print_text:
        print_messages(explain_messages)

    explanation_response = openai.ChatCompletion.create(
        engine=chatgpt_deployment_id,
        messages=explain_messages,
        temperature=temperature,
        stream=True,
    )
    explanation = concat_chunks_in_response(explanation_response)
    explain_assistant_message = {"role": "assistant", "content": explanation}
    return [explain_system_message, explain_user_message, explain_assistant_message]

def plan_with_detailed_natural_language(
    explain_system_message: str,
    explain_user_message: str,
    explain_assistant_message: str,
    ai_teacher_abilities: str,
    print_text: bool = True,
    temperature: float = 0.4
):
    plan_system_message = {
        "role": "system",
        "content": "You are a world-class Chinese teaching and research teacher with an eagle eye for learning objectives and student's deep-level requirements. You carefully create lesson plan which considering learning objectives and student's specific requirements with great detail and accuracy. You organize your lesson plan in markdown-formatted, bulleted lists."
    }
    plan_user_message = {
        "role": "user",
        "content": f"""
# High-Qulity Lesson Plan Standards
1. Perfectly meeting learing objectives
2. Perfectly meeting student's learning preference
3. Escaping teaching student's existing knowledge
4. Only using materials that the ai_teacher_abilities contains.
5. Providing diverse activities in each section
6. Providing attractive introduction
7. Splitting essay into sections should follow the logical point of the essay
9. Splitting essay into sections should aim for sections that can be meaningfully covered within a single lesson or class period, allowing for adequate time to explore the content and complete associated activities
10. Expressing content as activity only for introduction, unit-sections and integrative-sections 

# Input Parameters
## ai teacher abilities
{ai_teacher_abilities}

Create an 1-on-1 Lesson Plan step by step, and remember to follow "High-Qulity Lesson Plan Standards":
- Remind yourself with learing objectives, student's learning preference and student's existing knowledge
- Provide an introduction, following "High-Qulity Lesson Plan Standards" 1,2,3,6
- Divide the essay into smaller unit-sections, following "High-Qulity Lesson Plan Standards" 7,9
- Sequence the unit-sections, arrange the unit-sections in a logical order for teaching, follow the original structure of the essay or reorder the unit-sections based on the complexity of language features or thematic progression
- Provide a few integrative-sections to provide activities which cover the whole essay, such as "Summary and Synthesis", "Review and Reflection", "Conclusion" and so on.
- For each unit-section or integrative-section, following "High-Qulity Lesson Plan Standards" 10:
    * Create engaging activities in section, following "High-Qulity Lesson Plan Standards" 1,2,3,4,5
    * For each activity:
        1. Provide learning materials used (and mention material type) in each activity, following "High-Qulity Lesson Plan Standards" 4
""",
    }
    plan_messages = [
        plan_system_message,
        explain_user_message,
        explain_assistant_message,
        plan_user_message,
    ]
    if print_text:
        print_messages([plan_system_message, plan_user_message])
    plan_response = openai.ChatCompletion.create(
        engine=chatgpt_deployment_id,
        messages=plan_messages,
        temperature=temperature,
        stream=True,
    )
    plan = concat_chunks_in_response(plan_response)
    plan_assistant_message = {"role": "assistant", "content": plan}
    return [plan_user_message, plan_assistant_message]

def generate_executable_plan(
    explain_user_message: str,
    explain_assistant_message: str,
    plan_user_message: str,
    plan_assistant_message: str,
    available_materials: str,
    print_text: bool = True,
    temperature: float = 0.1
):
    # create a markdown-formatted prompt that asks GPT to complete a unit test
    execute_system_message = {
        "role": "system",
        "content": "You are a world-class Chinese teacher with an eagle eye for lesson plan. You write careful, accurate lesson plan. When asked to reply only with json, you write all of your code in a single block.",
    }
    execute_user_message = {
        "role": "user",
        "content": f"""Using json, write a lesson plan excuted by AI teacher for the student, following the text plan above. Enrich lesson plan's section with available materials if adapted. Reply only with json, using the following lesson_plan_schema:

# Output format
## lesson_plan_schema
```
{{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "properties": {{
    "lesson_plan": {{
      "type": "array",
      "items": {{
        "type": "object",
        "properties": {{
          "section_title": {{
            "type": "string"
          }},
          "steps": {{
            "type": "array",
            "items": {{
              "type": "object",
              "properties": {{
                "description": {{
                  "type": "string"
                }},
                "masterial_type": {{
                  "type": "string",
                  "enum": ["LLM_CHAT", "content_repo"]
                }},
                "chat_goal": {{
                  "type": "string",
                  "enum": ["yyy"]
                }},
                "material_ids": {{
                  "type": "array",
                  "items": {{
                    "type": "integer"
                  }}
                }}
              }},
              "required": ["description", "masterial_type"],
              "oneOf": [
                {{
                  "required": ["chat_goal"]
                }},
                {{
                  "required": ["material_ids"]
                }}
              ]
            }}
          }}
        }},
        "required": ["section_title", "steps"]
      }}
    }}
  }},
  "required": ["lesson_plan"]
}}
```

#Rules
1. For steps whose material type is content_repo, then the material_ids should be selected from available_materials
2. For other steps, material_type should be assigned to llm_chat, and should generate detailed chat_goal for this step

# Input Parameters
## available_materials
{available_materials}
""",
    }
    execute_messages = [
        execute_system_message,
        plan_assistant_message,
        execute_user_message
    ]
    if print_text:
        print_messages([execute_system_message, execute_user_message])

    execute_response = openai.ChatCompletion.create(
        engine=chatgpt_deployment_id,
        messages=execute_messages,
        temperature=temperature,
        stream=True,
    )
    execution = concat_chunks_in_response(execute_response)
    return execution

In [70]:
essay_content = """
### Essay Summary
《小马过河》讲述了一匹年轻的小马在妈妈的要求下去磨坊驮麦子的过程中，遇到了一条河。小马分别听到了牛和松鼠关于河水深浅的不同说法，犹豫不决，最后决定回家请教妈妈。经过妈妈的启发，小马明白了不能只听别人的话，要自己动脑筋，勇敢尝试。最后，小马顺利地过了河，完成了任务。这个故事教育孩子们要学会独立思考和勇敢尝试。

### Essay New words
"驮", "匹", "欣慰", "哗哗", "犹豫", "迈", "淹"
"""

student_requirement = """
### profile
我是一个小学3年级男生。
### requirements
我今天在课堂上学习过了《小马过河这篇文章》，但是我对这篇文章还没有理解，且有些生字也还不太会读写。
### prefered_format
我更喜欢通过游戏和讲故事的方式来学习。
"""

ai_teacher_abilities = """
### LLM Chat
- large language model's ability

### Existing Content For This Essay in Repository
- new word handwriting rating questions
- role play games
- read aloud rating questions
"""

available_materials = f"""* 5 new word handwriting questions, with material_ids=[101,102,103,104,105]
* 5 role play games, wtih material_ids=[201,202,203,204,205]
* 5 read aloud questions, wtih material_ids=[301,302,303,304,305]"""


[explain_system_message, explain_user_message, explain_assistant_message] = detailed_explanation_of_student_requirements(
    essay_content,
    student_requirement,
    print_text=True
)

[plan_user_message, plan_assistant_message] = plan_with_detailed_natural_language(
    explain_system_message, explain_assistant_message, explain_assistant_message, ai_teacher_abilities
)

# generate_executable_plan(
#     explain_user_message, explain_assistant_message, 
#     plan_user_message, plan_assistant_message, 
#     available_materials
# )



[system]
You are a world-class Chinese teaching and research teacher with an eagle eye for learning objectives and student's deep-level requirements. You carefully anlysis learning objectives and student's specific requirements with great detail and accuracy. You organize your analysis in markdown-formatted, bulleted lists.

[user]
Please explain learning objective and student's requirement in the following steps:
- Analyze the essay content.
- Explain the student's info.
- Try to conjecture student's existing knowledge in Chinese subject.
- Try to conjecture student's existing knowledge on given essay content.
- Analyze the student's potential deep-level requirements as much as possible based on previous steps.
- Define learning objectives: based on your above analysis, define clear and achievable learning objectives for this student.

Organize your explanation in English as a markdown-formatted, bulleted list.

# Input Parameters
## Essay Content

### Essay Summary
《小马过河》讲述了一匹年轻的小马在

1. Activity: Story Retelling
   - Have the student retell the story in their own words, summarizing the main events and lessons learned
   - Encourage the student to use the new vocabulary words they learned during the lesson
   - Use LLM Chat to help guide the student and provide feedback
   - Materials: None

## Integrative-Section 2: Review and Reflection
1. Activity: Personal Connection
   - Ask the student to think about a time when they faced a challenge or had to make a decision, and how they can apply the lessons from the story to their own life
   - Use LLM Chat to help guide the discussion and provide insights
   - Materials: None

## Conclusion
1. Review the learning objectives and discuss the student's progress during the lesson.
2. Congratulate the student on their hard work and encourage them to continue practicing their Chinese reading, writing, and critical thinking skills.
3. End the lesson with a fun question, such as "If you were the young horse, what would you do di

Make sure to check any code before using it, as GPT makes plenty of mistakes (especially on character-based tasks like this one). For best results, use the most powerful model (GPT-4, as of May 2023).